# Formulación de un problema de optimización

Para esta segunda parte hemos decidido implementar un problema de optimización que busca obtener la vuelta más rapida de clasificación de la Formula 1. Para ellos hemos tenido en cuenta algunos parametros del pilote y del coche. 

El circuito escogido para las pruebas es el de Bahrain, el cual hemos dividido en sectores. Para cada sector hemos cogido su longitud, velocidad media de ese sector y el tipo de sector ( curva o recta ). Todos los datos recogidos son tomados con la mayor precisión, la distancia real ( en metros ) y las velocidades medias ( en km/h) de los pilotos en ese orden. 

In [1]:
circuit =  [(1100,333.134,'s'), ( 94.53, 100, 't') , (72.38,193.121,'t'), (132.12,225.308,'t'),(491.22,278.417,'s'),
            (151.97,152.888,'t'),(183.74,229.33,'s'),(116.64,225.308,'t'),(123.55,249.448,'t'),(181.68,270.37,'s'),
            (69.69,112.654,'t'),(339.00,211.628,'s'),(103.32,120.701,'t'),(655.09,255.08,'s'),(313.49,209.215,'t'),
            (214.69,280.026,'t'),(236.00,211.88,'t'),(713.44,257.495,'s'),(110.87,162.554,'t')]

La función objetivo calcula el tiempo de vuelta para los parametros ("attributes"). Los parametros que se reciben son:
    
    -driver_tyres_treatment:
    Se corresponde con la capacidad de piloto de cuidar sus ruedas. Los valores que puede tomar son de 0.00 a 10.00. Un mejor cuidado de las ruedas sopone una perdida de velocidad (entre 0.00/+2.00%) a lo largo de la vuelta pero una menor probabilidad de cometer un error ("fail_p", entre un 0.00/+10 %)
    
    -driver_agro:
    Se corresponde con la agresividad del piloto. Los valores que puede tomar son de 0.00 a 10.00. Un piloto más agresivo irá más rapido (entre un 0.00 y un 1.00%) pero se arriesgará a cometer más errores ("fail_p", entre un 0.00/+2.00%) o chocar ("crash_p", entre un 0.00/+2.00%).
    
    -driver_experience:
    Se corresponde con la experiencia del piloto. Los valores que puede tomar son de 0.00 a 10.00. Un piloto más experimentado cometerá menos errores ("fail_p", entre el -1.00/+4.00% ) pero por ende irá más lento  o chocar ("crash_p",entre un -0.25/+2.00%).

    -car_motor:
    Se corresponde con la potencia del motor. Los valores que puede tomar son de 0.00 a 10.00. Un coche con un motor más potente correrá más (entre un 0.00/+4.00%) en las rectas, pero pesará más haciendolo más lento en general (entre un 0.00/+1.00%).

    -car_aerodynamic_load:
    Se corresponde con la carga aerodinámica del coche. Los valores que puede tomar son de 0.00 a 10.00. Un coche con una mejor carga aerodinámica irá más rápido en curvas (entre un 0.00/+4.00%) pero más lento en rectas (entre un 0.00/+2.00%).

    -car_reliability:
    Se corresponde con la fiabilidad del coche. Los valores que pueden tomar son de 0.00 a 10.00. Un coche con una mayor fiabilidad será más lento (entre un 0.00/+1.00%) pero menos propenso a sufrir fallos ("fail_p", entre un -1.00/+1.00) y posibles choques ("crash_p", entre un -0.25/+0.25).

        
        
        
        
 Finalmente, calculamos los porcentajes que aplicarán a las velocidades de los sectores ("bonus_global""bonus_turns","bonus_straight", "fail_p", "crash_p"). Seguidamente recorremos el circuito sector a sector. Si     el sector es un recta se multpilica su velocidad media por la media de los "bonus_global" y "bonus_straight". Si el sector es   una cruva se multiplica su velocidad media por la media de los "bonus_global" y "bonus_turns". Una vez recalculado las           velocidades se calcula el tiempo que se tarda en atravesar dicho sector. Además en cada sector hay la posibilidad de cometer     un fallo o chocar. En caso de cometer un fallo se penalizaria con 1.5 segudos en el sector. En caso de chocar no será posible   completar la vuelta por lo que el tiempo global será infinito.
    

In [2]:
import random

def fitness_func(attributes):
    bonus_turns = 100
    bonus_straight = 100
    bonus_global = 100
    fail_p = 0
    crash_p = 0
    
    time_global = 0.00
    
    driver_tyres_treatment = attributes[0] / 10.00
    driver_agro = attributes [1] / 10.00
    driver_experience = attributes [2] / 10.00
    car_motor = attributes[3] / 10.00
    car_aerodynamic_load = attributes[4] / 10.00
    car_reliability = attributes[5] / 10.00
        
    #tyres_treatment balance
    bonus_global += -driver_tyres_treatment * 2.00
    fail_p += (10.00 - driver_tyres_treatment*10.00) 
    
    #agro balance
    bonus_global += driver_agro * 1.00 + ( -1.00 )
    fail_p += driver_agro * 10.00
    crash_p += driver_agro * 2.00
    
    #experience balance
    bonus_global += -driver_experience* 2.00
    fail_p += 4.00 - driver_experience * 5.00
    crash_p += 0.25 - driver_experience * 2.25
    
    #reliability balance
    bonus_global += -car_reliability
    fail_p += car_reliability * 2.00 + ( -1.00 )
    crash_p += car_reliability * 0.50 + ( -0.25 )
    
    #motor balance
    bonus_global += - car_motor*1.00 
    bonus_straight += car_motor * 4.00 - ( 4.00 ) 
    
    #aerodynamic_load balance
    bonus_straight += - car_aerodynamic_load*2.00 
    bonus_turns += car_aerodynamic_load *4.00 - (4.00)
    
    for sector in circuit:
        m_per_seg = sector[1] * 1000/3600
        time_sector = 0
        
        if (sector[2] == 's'):
            m_per_seg *= ((bonus_global + bonus_straight)/2) / 100
            time_sector += sector[0] / m_per_seg
        else:
            m_per_seg *= ((bonus_global + bonus_turns)/2) / 100
            time_sector += sector[0] / m_per_seg
        
        if random.random() < fail_p / 100:
            time_sector += 1.5 
        if random.random() < crash_p / 100:
            time_global = float("+inf")
            return time_global
        
        
        time_global += time_sector
        
    return time_global
     
   

Ahora ejecutamos un DE de biblioteca con nuestra función objetivo y un tamaño de población de 50 y 10.000 iteraciones.

In [3]:
from scipy.optimize import differential_evolution

limites = [(0.00,10.00)]*6
time = differential_evolution(fitness_func,limites, popsize=50, polish=False, maxiter=10000)

In [4]:
import datetime
print("Cuidado de neumaticos: " +str(time.x[0]))
print("Agresividad piloto: " +str(time.x[1]))
print("Experiencia piloto: " +str(time.x[2]))
print("Motor: " +str(time.x[3]))
print("Carga aerodinamica: " +str(time.x[4]))
print("Fiabilidad: " +str(time.x[5]))
print("------------------------------")
print("Tiempo de vuelta: "+str(datetime.timedelta(seconds=time.fun)))


Cuidado de neumaticos: 0.6443105277198846
Agresividad piloto: 5.0380723322174426
Experiencia piloto: 1.2923464525757957
Motor: 8.989042282150265
Carga aerodinamica: 2.765185484601621
Fiabilidad: 1.7052964527251975
------------------------------
Tiempo de vuelta: 0:01:24.106872


Como podemos ver, en el circuito de Bahrain el motor es bastante importante debido a sus largas rectas. Además no se necesita mucha carga aerodinámica. La fiabilidad y el cuidado de los neumáticos no son muy necesarios para poder hacer la vuelta más rápida.
En cuanto al piloto lo mejor es que no sea muy agresivo ni muy conservador.
El tiempo obtenido es muy bueno, por lo tanto ya sabemos la configuración adecuada para batir la vuelta rápida de Pedro De La Rosa.